# GREAT CARIA - Multi-Scale Fragility with Temporal Relativity

## Core Framework:

**Proposition 1**: Fragility emerges from structural factors at multiple temporal scales

**Proposition 2**: A system can appear stable short-term while hiding long-term tensions

**Proposition 3**: Psychology is an AMPLIFIER, not a cause - weights adjusted accordingly

---

## Temporal Scales:
| Scale | Horizon | Agents |
|-------|---------|--------|
| Ultra-Fast | <1 day | HFT, algorithms |
| Short | 1-10 days | Traders, retail |
| Medium | 10-60 days | Hedge funds, tactical |
| Long | 60-250 days | Institutions |
| Ultra-Long | >250 days | Central banks, pensions |

In [ ]:
!pip install PyWavelets -q

import pandas as pd
import numpy as np
from scipy import stats, signal
from scipy.ndimage import gaussian_filter1d
import pywt
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_parquet('/content/drive/MyDrive/CARIA/data/raw/yahoo_market.parquet')
COUNTRIES = ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'FRA', 'BRA', 'MEX', 'KOR', 'AUS', 'IND', 'ZAF']
idx_cols = [f'{c}_index' for c in COUNTRIES if f'{c}_index' in df.columns]
ret = df[idx_cols].pct_change().dropna()
ret.columns = [c.replace('_index', '') for c in ret.columns]
print(f'Data: {ret.shape}, {ret.index.min().date()} to {ret.index.max().date()}')

In [ ]:
# === Crisis Catalog ===
CRISES = {
    'Lehman': pd.Timestamp('2008-09-15'),
    'Flash_Crash': pd.Timestamp('2010-05-06'),
    'Euro_Crisis': pd.Timestamp('2011-08-05'),
    'Taper_Tantrum': pd.Timestamp('2013-05-22'),
    'China_Crash': pd.Timestamp('2015-08-24'),
    'Brexit': pd.Timestamp('2016-06-24'),
    'Volmageddon': pd.Timestamp('2018-02-05'),
    'COVID': pd.Timestamp('2020-03-11'),
    'Gilt_Crisis': pd.Timestamp('2022-09-23'),
    'SVB': pd.Timestamp('2023-03-10')
}
data_start = ret.index.min()
CRISES = {k: v for k, v in CRISES.items() if v > data_start + pd.Timedelta(days=300)}
print(f'Crises in range: {len(CRISES)}')

---
# PART 1: 5-SCALE TEMPORAL DECOMPOSITION

Using moving average filters for interpretable scale separation

In [ ]:
# === 1A: Base Crisis Factor ===
def compute_cf(r, w=20):
    cf = []
    for i in range(w, len(r)):
        wr = r.iloc[i-w:i]
        c = wr.corr().values
        ac = (c.sum() - len(c)) / (len(c) * (len(c) - 1))
        cf.append(ac * wr.std().mean() * 100)
    return pd.Series(cf, index=r.index[w:])

CF = compute_cf(ret)
print(f'CF computed: {len(CF)}')

In [ ]:
# === 1B: 5-Scale Decomposition ===
print('=== 5-Scale Temporal Decomposition ===')

SCALES = {
    'ultra_fast': {'window': 1, 'description': '<1 day (HFT noise)'},
    'short': {'window': 5, 'description': '1-10 days (traders)'},
    'medium': {'window': 30, 'description': '10-60 days (tactical)'},
    'long': {'window': 120, 'description': '60-250 days (institutions)'},
    'ultra_long': {'window': 252, 'description': '>250 days (macro cycle)'}
}

def decompose_scales(series, scales):
    """Decompose into 5 scales using cascading moving averages"""
    bands = {}
    residual = series.copy()
    
    sorted_scales = sorted(scales.items(), key=lambda x: x[1]['window'])
    
    for i, (name, config) in enumerate(sorted_scales):
        w = config['window']
        smooth = residual.rolling(w, min_periods=1).mean()
        
        if i < len(sorted_scales) - 1:
            # This band = current smooth - next smooth
            next_w = sorted_scales[i+1][1]['window']
            next_smooth = residual.rolling(next_w, min_periods=1).mean()
            bands[name] = smooth - next_smooth
        else:
            # Last band = remaining trend
            bands[name] = smooth
    
    return bands

CF_scales = decompose_scales(CF, SCALES)

for name, config in SCALES.items():
    print(f"  {name}: {config['description']}")

In [ ]:
# === 1C: Compute indicators at each scale ===
print('\n=== Indicators per Scale ===')

def compute_scale_indicators(band, window=60):
    """Compute variance, ACF1, skewness for each scale"""
    return pd.DataFrame({
        'variance': band.rolling(window).var(),
        'acf1': band.rolling(window).apply(lambda x: x.autocorr(1) if len(x) > 1 else 0, raw=False),
        'skewness': band.rolling(window).skew()
    })

scale_indicators = {}
for name, band in CF_scales.items():
    scale_indicators[name] = compute_scale_indicators(band)
    print(f"  {name}: computed")

In [ ]:
# === 1D: Visualize scales ===

fig, axes = plt.subplots(6, 1, figsize=(14, 18), sharex=True)

# Full CF
axes[0].plot(CF.index, CF.values, 'k-', linewidth=0.5)
axes[0].set_ylabel('CF (full)')
axes[0].set_title('Crisis Factor: 5-Scale Temporal Decomposition', fontsize=14)

# Each scale
colors = {'ultra_fast': 'red', 'short': 'orange', 'medium': 'gold', 
          'long': 'blue', 'ultra_long': 'purple'}

for i, (name, band) in enumerate(CF_scales.items()):
    ax = axes[i+1]
    ax.plot(band.index, band.values, color=colors[name], linewidth=0.8)
    ax.set_ylabel(name)
    ax.set_title(f"{SCALES[name]['description']}", fontsize=10)
    
    # Mark crises
    for date in CRISES.values():
        ax.axvline(date, color='gray', alpha=0.3, linestyle=':')

axes[0].legend([plt.Line2D([0], [0], color='gray', linestyle=':')], ['Crises'])
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/CARIA/research/formal_fragility/5_scale_decomposition.png', dpi=150)
plt.show()

---
# PART 2: LEAD TIME ANALYSIS AND WEIGHT CALIBRATION

Weights inversely proportional to lead time

In [ ]:
# === 2A: Compute lead time for each scale ===
print('=== Lead Time Analysis per Scale ===')

def compute_lead_time(indicator, crisis_date, threshold_pct=0.8, lookback=180):
    """Days before crisis that indicator crossed threshold"""
    threshold = indicator.quantile(threshold_pct)
    pre = indicator[(indicator.index < crisis_date) & 
                    (indicator.index > crisis_date - pd.Timedelta(days=lookback))]
    crossings = pre[pre > threshold]
    if len(crossings) > 0:
        return (crisis_date - crossings.index[0]).days
    return 0

# Compute lead times for variance at each scale
lead_times = {scale: [] for scale in SCALES.keys()}

for crisis_name, crisis_date in CRISES.items():
    for scale in SCALES.keys():
        var = scale_indicators[scale]['variance'].dropna()
        if len(var) > 0 and crisis_date > var.index.min() + pd.Timedelta(days=180):
            lead = compute_lead_time(var, crisis_date)
            lead_times[scale].append(lead)

# Average lead times
avg_leads = {scale: np.mean(leads) if leads else 0 for scale, leads in lead_times.items()}

print('\nAverage Lead Times (days):')
for scale, lead in sorted(avg_leads.items(), key=lambda x: -x[1]):
    print(f"  {scale:12s}: {lead:6.1f} days")

In [ ]:
# === 2B: Compute weights inversely proportional to lead time ===
print('\n=== Weight Calibration (1/lead_time) ===')

# Avoid division by zero
min_lead = 1  # Minimum 1 day
adjusted_leads = {k: max(v, min_lead) for k, v in avg_leads.items()}

# Inverse weights (longer lead = more weight)
raw_weights = {k: v for k, v in adjusted_leads.items()}
total = sum(raw_weights.values())
scale_weights = {k: v / total for k, v in raw_weights.items()}

print('\nScale Weights (based on lead time):')
for scale, weight in sorted(scale_weights.items(), key=lambda x: -x[1]):
    print(f"  {scale:12s}: {weight:.3f} (lead={avg_leads[scale]:.0f}d)")

---
# PART 3: PSYCHOLOGY AS AMPLIFIER (Reduced Weights)

Synchronization and crowding are late signals - reduce their weight

In [ ]:
# === 3A: Compute synchronization (Kuramoto) ===
print('=== Psychology Indicators (Amplifiers) ===')

def kuramoto_sync(returns, window=60):
    phases = pd.DataFrame({
        c: np.angle(signal.hilbert(returns[c].fillna(0) - 
                                   gaussian_filter1d(returns[c].fillna(0).values, 60)))
        for c in returns.columns
    }, index=returns.index)
    
    sync = [np.abs(np.exp(1j * phases.iloc[i].values).mean()) 
            for i in range(window, len(phases))]
    return pd.Series(sync, index=phases.index[window:])

SYNC = kuramoto_sync(ret)

# Lead time for sync
sync_leads = []
for crisis_date in CRISES.values():
    if crisis_date > SYNC.index.min() + pd.Timedelta(days=180):
        lead = compute_lead_time(SYNC, crisis_date)
        sync_leads.append(lead)

avg_sync_lead = np.mean(sync_leads) if sync_leads else 0
print(f'Synchronization avg lead: {avg_sync_lead:.1f} days')
print('→ Typically shorter than structural signals → AMPLIFIER role confirmed')

In [ ]:
# === 3B: Adjusted weights for psychology ===

# Psychology weight based on its lead time
# Since it's typically late, it gets lower weight
total_structural_lead = sum(avg_leads.values())
psych_weight_raw = avg_sync_lead
total_lead = total_structural_lead + psych_weight_raw

# Final weight allocation
STRUCTURAL_WEIGHT = total_structural_lead / total_lead  # ~0.85-0.95
PSYCHOLOGY_WEIGHT = psych_weight_raw / total_lead       # ~0.05-0.15

print(f'\nWeight Allocation:')
print(f'  Structural factors: {STRUCTURAL_WEIGHT:.1%}')
print(f'  Psychology (amplifier): {PSYCHOLOGY_WEIGHT:.1%}')

# Redistribute structural weight to scales
final_scale_weights = {k: v * STRUCTURAL_WEIGHT for k, v in scale_weights.items()}
final_scale_weights['psychology'] = PSYCHOLOGY_WEIGHT

---
# PART 4: MULTI-SCALE FRAGILITY INDEX

In [ ]:
# === 4A: Normalize all signals ===
print('=== Constructing Multi-Scale Fragility Index ===')

# Align all signals
common_idx = CF.index
for scale in SCALES.keys():
    common_idx = common_idx.intersection(scale_indicators[scale]['variance'].dropna().index)
common_idx = common_idx.intersection(SYNC.dropna().index)

# Standardize each scale's variance
scaler = StandardScaler()
normalized = {}

for scale in SCALES.keys():
    var = scale_indicators[scale]['variance'].loc[common_idx]
    normalized[scale] = pd.Series(
        scaler.fit_transform(var.values.reshape(-1, 1)).flatten(),
        index=common_idx
    )

# Psychology (sync)
normalized['psychology'] = pd.Series(
    scaler.fit_transform(SYNC.loc[common_idx].values.reshape(-1, 1)).flatten(),
    index=common_idx
)

print(f'Normalized signals: {len(normalized)} components')

In [ ]:
# === 4B: Combine with calibrated weights ===

# Final weights
print('\nFinal Weights:')
for name, weight in sorted(final_scale_weights.items(), key=lambda x: -x[1]):
    role = 'AMPLIFIER' if name == 'psychology' else 'STRUCTURAL'
    print(f"  {name:12s}: {weight:.3f} ({role})")

# Compute index
MSFI = sum(normalized[name] * weight for name, weight in final_scale_weights.items())

# Normalize to 0-1 range
MSFI = (MSFI - MSFI.min()) / (MSFI.max() - MSFI.min())

print(f'\nMSFI computed: {len(MSFI)} samples')
print(f'Range: {MSFI.min():.3f} - {MSFI.max():.3f}')

In [ ]:
# === 4C: Visualize final index ===

fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True)

# MSFI
axes[0].fill_between(MSFI.index, MSFI.values, alpha=0.3, color='red')
axes[0].plot(MSFI.index, MSFI.values, 'r-', linewidth=0.8)
axes[0].axhline(MSFI.quantile(0.8), color='orange', linestyle='--', label='Warning (80%)')
axes[0].axhline(MSFI.quantile(0.95), color='darkred', linestyle='--', label='Critical (95%)')
axes[0].set_ylabel('MSFI')
axes[0].set_title('Multi-Scale Fragility Index', fontsize=14)
axes[0].legend()

# Structural component
structural = sum(normalized[s] * final_scale_weights[s] for s in SCALES.keys())
axes[1].plot(structural.index, structural.values, 'b-', linewidth=0.8)
axes[1].set_ylabel('Structural')
axes[1].set_title(f'Structural Component ({STRUCTURAL_WEIGHT:.0%} weight)')

# Psychology component
psychology = normalized['psychology'] * final_scale_weights['psychology']
axes[2].plot(psychology.index, psychology.values, 'purple', linewidth=0.8)
axes[2].set_ylabel('Psychology')
axes[2].set_title(f'Psychology Amplifier ({PSYCHOLOGY_WEIGHT:.0%} weight)')

# S&P 500
sp500 = df['USA_index'].loc[MSFI.index].dropna()
axes[3].plot(sp500.index, sp500.values, 'k-', linewidth=0.5)
axes[3].set_ylabel('S&P 500')
axes[3].set_yscale('log')

# Mark crises
for ax in axes:
    for name, date in CRISES.items():
        ax.axvline(date, color='gray', alpha=0.4, linestyle=':')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/CARIA/research/formal_fragility/msfi_final.png', dpi=150)
plt.show()

---
# PART 5: RELATIVE TEMPORAL PERCEPTION

Comparing G7 vs Emerging Markets

In [ ]:
# === 5A: Country groups ===
print('=== Temporal Relativity: G7 vs Emerging ===')

G7 = ['USA', 'JPN', 'DEU', 'GBR', 'FRA']
EM = ['CHN', 'BRA', 'MEX', 'KOR', 'IND', 'ZAF']

g7_cols = [c for c in ret.columns if c in G7]
em_cols = [c for c in ret.columns if c in EM]

# Compute CF for each group
CF_G7 = compute_cf(ret[g7_cols])
CF_EM = compute_cf(ret[em_cols])

# Decompose each
G7_scales = decompose_scales(CF_G7, SCALES)
EM_scales = decompose_scales(CF_EM, SCALES)

print(f'G7 countries: {g7_cols}')
print(f'EM countries: {em_cols}')

In [ ]:
# === 5B: Compare lead times by group ===

group_leads = {'G7': {}, 'EM': {}}

for scale in SCALES.keys():
    # G7
    g7_var = G7_scales[scale].rolling(60).var().dropna()
    g7_leads = []
    for crisis_date in CRISES.values():
        if crisis_date > g7_var.index.min() + pd.Timedelta(days=180):
            lead = compute_lead_time(g7_var, crisis_date)
            g7_leads.append(lead)
    group_leads['G7'][scale] = np.mean(g7_leads) if g7_leads else 0
    
    # EM
    em_var = EM_scales[scale].rolling(60).var().dropna()
    em_leads = []
    for crisis_date in CRISES.values():
        if crisis_date > em_var.index.min() + pd.Timedelta(days=180):
            lead = compute_lead_time(em_var, crisis_date)
            em_leads.append(lead)
    group_leads['EM'][scale] = np.mean(em_leads) if em_leads else 0

# Display
print('\nLead Times by Group (days):')
print(f'{"Scale":12s} | {"G7":>8s} | {"EM":>8s} | {"Diff":>8s}')
print('-' * 45)
for scale in SCALES.keys():
    g7 = group_leads['G7'][scale]
    em = group_leads['EM'][scale]
    diff = g7 - em
    print(f'{scale:12s} | {g7:8.1f} | {em:8.1f} | {diff:+8.1f}')

print('\n→ Different temporal perception = TEMPORAL RELATIVITY confirmed')

In [ ]:
# === 5C: Visualize temporal relativity ===

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Long scale comparison
g7_long = G7_scales['long'].rolling(60).var()
em_long = EM_scales['long'].rolling(60).var()

axes[0].plot(g7_long.index, g7_long.values, 'b-', label='G7', alpha=0.8)
axes[0].plot(em_long.index, em_long.values, 'orange', label='EM', alpha=0.8)
axes[0].set_ylabel('Variance (long scale)')
axes[0].set_title('Temporal Relativity: G7 vs Emerging Markets (60-250d scale)')
axes[0].legend()

# Lead difference
lead_diff = g7_long - em_long
axes[1].fill_between(lead_diff.index, lead_diff.values, 0, 
                     where=lead_diff > 0, alpha=0.5, color='blue', label='G7 higher')
axes[1].fill_between(lead_diff.index, lead_diff.values, 0,
                     where=lead_diff <= 0, alpha=0.5, color='orange', label='EM higher')
axes[1].set_ylabel('G7 - EM')
axes[1].set_title('Which group perceives risk earlier?')
axes[1].legend()

for ax in axes:
    for date in CRISES.values():
        ax.axvline(date, color='gray', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/CARIA/research/formal_fragility/temporal_relativity.png', dpi=150)
plt.show()

---
# PART 6: BIFURCATION DETECTION (Multi-Scale Instability)

In [ ]:
# === 6A: Detect when multiple scales are unstable ===
print('=== Bifurcation Detection ===')

def multi_scale_instability(scale_indicators, threshold_pct=0.8):
    """Count how many scales are above their instability threshold"""
    instability_count = pd.Series(0, index=common_idx)
    
    for scale in SCALES.keys():
        var = scale_indicators[scale]['variance'].loc[common_idx]
        threshold = var.quantile(threshold_pct)
        instability_count += (var > threshold).astype(int)
    
    return instability_count

instability_count = multi_scale_instability(scale_indicators)

print(f'Instability count range: {instability_count.min()} - {instability_count.max()} scales')

In [ ]:
# === 6B: Bifurcation warning ===

# Bifurcation = 3+ scales unstable simultaneously
BIFURCATION_THRESHOLD = 3
bifurcation_warning = instability_count >= BIFURCATION_THRESHOLD

print(f'Bifurcation warnings: {bifurcation_warning.sum()} days ({bifurcation_warning.mean():.1%})')

# Check pre-crisis
print('\nPre-crisis bifurcation status (60d before):')
for crisis_name, crisis_date in CRISES.items():
    if crisis_date > instability_count.index.min():
        pre = instability_count[(instability_count.index < crisis_date) & 
                                (instability_count.index > crisis_date - pd.Timedelta(days=60))]
        avg_unstable = pre.mean()
        had_warning = (pre >= BIFURCATION_THRESHOLD).any()
        print(f"  {crisis_name}: avg {avg_unstable:.1f} scales unstable, warning={had_warning}")

In [ ]:
# === 6C: Final visualization ===

fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# MSFI
axes[0].fill_between(MSFI.index, MSFI.values, alpha=0.3, color='red')
axes[0].plot(MSFI.index, MSFI.values, 'r-', linewidth=0.8)
axes[0].set_ylabel('MSFI')
axes[0].set_title('Multi-Scale Fragility Index with Bifurcation Detection')

# Instability count
axes[1].fill_between(instability_count.index, instability_count.values, alpha=0.5, color='purple')
axes[1].axhline(BIFURCATION_THRESHOLD, color='red', linestyle='--', label=f'Bifurcation ({BIFURCATION_THRESHOLD}+ scales)')
axes[1].set_ylabel('Unstable Scales')
axes[1].set_title('Multi-Scale Instability Count')
axes[1].legend()

# S&P 500 with bifurcation markers
sp500 = df['USA_index'].loc[MSFI.index].dropna()
axes[2].plot(sp500.index, sp500.values, 'k-', linewidth=0.5)
# Mark bifurcation periods
bif_periods = bifurcation_warning[bifurcation_warning].index
for date in bif_periods[::5]:  # Sample to avoid clutter
    if date in sp500.index:
        axes[2].axvspan(date, date + pd.Timedelta(days=1), alpha=0.3, color='red')
axes[2].set_ylabel('S&P 500')
axes[2].set_yscale('log')
axes[2].set_title('S&P 500 with Bifurcation Warnings (red)')

for ax in axes:
    for date in CRISES.values():
        ax.axvline(date, color='blue', alpha=0.4, linestyle=':')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/CARIA/research/formal_fragility/bifurcation_detection.png', dpi=150)
plt.show()

In [ ]:
# === FINAL SUMMARY ===
print('\n' + '='*70)
print('GREAT CARIA - MULTI-SCALE FRAGILITY WITH TEMPORAL RELATIVITY')
print('='*70)

print('\n📊 TEMPORAL SCALES:')
for scale, config in SCALES.items():
    lead = avg_leads.get(scale, 0)
    weight = final_scale_weights.get(scale, 0)
    print(f"  {scale:12s}: {config['description']:25s} lead={lead:5.0f}d, weight={weight:.3f}")

print(f"\n🧠 PSYCHOLOGY (AMPLIFIER):")
print(f"  Weight: {PSYCHOLOGY_WEIGHT:.1%} (reduced due to short lead time)")
print(f"  Role: Confirms structural signals, does not initiate")

print(f"\n🌍 TEMPORAL RELATIVITY:")
print(f"  G7 avg lead: {np.mean(list(group_leads['G7'].values())):.0f} days")
print(f"  EM avg lead: {np.mean(list(group_leads['EM'].values())):.0f} days")
print(f"  → Each group perceives risk in different 'temporal dimension'")

print(f"\n🌀 BIFURCATION DETECTION:")
print(f"  Threshold: {BIFURCATION_THRESHOLD}+ scales unstable")
print(f"  Total warnings: {bifurcation_warning.sum()} days")

print('\n' + '='*70)

In [ ]:
# === EXPORT ===
import json

def safe_serialize(obj):
    if hasattr(obj, 'isoformat'):
        return obj.isoformat()
    elif isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, dict):
        return {str(k): safe_serialize(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [safe_serialize(i) for i in obj]
    return obj

export = {
    'version': 'Great Caria Multi-Scale Fragility v2.0',
    'generated': pd.Timestamp.now().isoformat(),
    'methodology': {
        'scales': SCALES,
        'weights': safe_serialize(final_scale_weights),
        'structural_weight': float(STRUCTURAL_WEIGHT),
        'psychology_weight': float(PSYCHOLOGY_WEIGHT),
        'bifurcation_threshold': BIFURCATION_THRESHOLD
    },
    'lead_times': safe_serialize(avg_leads),
    'temporal_relativity': {
        'G7': safe_serialize(group_leads['G7']),
        'EM': safe_serialize(group_leads['EM'])
    },
    'thresholds': {
        'warning': float(MSFI.quantile(0.8)),
        'critical': float(MSFI.quantile(0.95))
    },
    'current': {
        'msfi': float(MSFI.iloc[-1]),
        'structural': float(structural.iloc[-1]),
        'psychology': float(psychology.iloc[-1]),
        'unstable_scales': int(instability_count.iloc[-1]),
        'bifurcation_warning': bool(bifurcation_warning.iloc[-1])
    },
    'crises_validated': len(CRISES)
}

OUTPUT_DIR = '/content/drive/MyDrive/CARIA/research/formal_fragility'
with open(f'{OUTPUT_DIR}/multiscale_fragility_v2.json', 'w') as f:
    json.dump(export, f, indent=2)

print('\n✓ Exported: multiscale_fragility_v2.json')
print(f'\nSaved figures:')
print('  5_scale_decomposition.png')
print('  msfi_final.png')
print('  temporal_relativity.png')
print('  bifurcation_detection.png')